# Visualize Data

In [ ]:
%load_ext autoreload
%autoreload 2
from data_loader_h5 import H5Dataset
from data_loader_jsonl import JSONLDataset

dataset_location = "/tmp/clevr-act-7-depth"
train_dataset = H5Dataset(dataset_location)

#train_dataset_jsonl = JSONLDataset(dataset_location)
print(len(train_dataset))

In [ ]:
from IPython.display import display, HTML
from tqdm.notebook import tqdm
from utils_vis import render_example

print(len(train_dataset))
num_samples = 2
html_imgs = ""
for i in tqdm(range(num_samples)):
    image, sample = train_dataset[i]
    html_imgs += render_example(image, label=sample["suffix"], text=sample["prefix"], camera=sample["camera"])    
display(HTML(html_imgs))

# ManiSkill H5 Datasets

In [ ]:
import glob
from pathlib import Path
import h5py

def print_h5_structure(name, obj):
    """
    Prints the path, type, and shape of each member in an h5 file.
    """
    # Check if the object is a dataset
    if isinstance(obj, h5py.Dataset):
        print(f"Dataset: {name} | Shape: {obj.shape} | Type: {obj.dtype}")
    # If it's a group, just print its name
    elif isinstance(obj, h5py.Group):
        print(f"Group: {name}")

traj_path = Path("/data/lmbraid19/argusm/datasets/clevr-act-9-ms-small/20250205_182607.h5")
idx = 0

with h5py.File(traj_path, "r") as h5_file:
    print(h5_file['traj_0/obs/sensor_param/render_camera'].keys())
    print_h5_structure(f"traj_{idx}", h5_file)

In [ ]:
def h5_tree(val, pre=''):
    items = len(val)
    for key, val in val.items():
        items -= 1
        if items == 0:
            # the last item
            if type(val) == h5py._hl.group.Group:
                print(pre + '└── ' + key)
                h5_tree(val, pre+'    ')
            else:
                try:
                    print(pre + '└── ' + key + ' (%d)' % len(val))
                except TypeError:
                    print(pre + '└── ' + key + ' (scalar)')
        else:
            if type(val) == h5py._hl.group.Group:
                print(pre + '├── ' + key)
                h5_tree(val, pre+'│   ')
            else:
                try:
                    print(pre + '├── ' + key + ' (%d)' % len(val))
                except TypeError:
                    print(pre + '├── ' + key + ' (scalar)')

h5_tree(train_dataset.h5_file)